In [47]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import gzip
import os
import sys
import time

import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

In [68]:
dt = pd.read_csv('FLIC_dataset.csv', nrows = 300)
del dt['Unnamed: 0']

In [69]:
# To evaluate how good a prediction is
def eval_error(pred, ground_truth):
    return np.sqrt(mean_squared_error(pred, ground_truth))

In [72]:
y_x = y.ix[:,0:9] / 720

In [76]:
y_y = y.ix[:,9:] / 480

In [77]:
y = pd.concat([y_x,y_y], axis = 1)

In [78]:
x = dt['Image'].apply(lambda img:np.fromstring(img, sep=' '))

In [79]:
x2 = np.matrix(x.tolist())

In [80]:
x2.shape

(300, 345600)

In [81]:
from sklearn.cross_validation import train_test_split
x_train, x_valid, y_train,y_valid = train_test_split(x2, y, test_size = 0.2, random_state=0)

In [ ]:
# Split dt into train and valid
feature_names = dt.columns[0:-1]
y = dt[feature_names]
y = y.as_matrix()
x = dt['Image'].apply(lambda img:np.fromstring(img, sep=' '))
x = x.as_matrix()
from sklearn.cross_validation import train_test_split
x_train, x_valid, y_train,y_valid = train_test_split(x, y, test_size = 0.2, random_state=0)

In [7]:
### -------------------------------------------------------------------
### Start TF graph

image_height = 480
image_width = 720
batch_size = 32
num_labels = 18 # We are predicting 18 coordinates values - 9 points
num_channels = 4 # 4 channels in convolution

deep_grap = tf.Graph()
with deep_grap.as_default():
    tf_train_x = tf.placeholder(tf.float32, shape=(batch_size, image_height, image_width, num_channels))
    tf_train_y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_x = tf.constant(x_valid)
    
    conv1_weight = tf.Variable(
                            tf.truncated_normal([5,5, num_channels, 32], 
                                                stddev=0.1, seed=0, dtype=tf.float32))
    conv1_biases = tf.Variable(tf.zeros([32]))
    
    conv2_weights = tf.Variable(
        tf.truncated_normal([5, 5, 32, 64], 
                            stddev = 0.1, seed = 0, dtype = tf.float32))
    
    conv2_biases = tf.Variable(tf.constant(0.1, shape = [64]))
    
    fc1_weights = tf.Variable(
        tf.truncated_normal([image_height * image_width * 16 , 512], 
                            stddev = 0.1, seed = 0, dtype = tf.float32))

    fc1_biases = tf.Variable(tf.constant(0.1, shape = [512]))

    fc2_weights = tf.Variable(
        tf.truncated_normal([512, 512], 
                            stddev = 0.1, seed = 0, dtype = tf.float32))

    fc2_biases = tf.Variable(tf.constant(0.1, shape = [512]))

    fc3_weights = tf.Variable(
        tf.truncated_normal([512, num_labels], stddev = 0.1, seed = 0, dtype = tf.float32))

    fc3_biases = tf.Variable(tf.constant(0.1, shape = [num_labels]))
    def model_part1(data, train = False):
        
        conv1 = tf.nn.conv2d(data, conv1_weight, strides = [1,1,1,1], padding = 'SAME')

        relu1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_biases))

        pool1 = tf.nn.max_pool(relu1, ksize = [1, 2, 2, 1], 
                                strides = [1, 2, 2, 1], padding = 'SAME')

        conv2 = tf.nn.conv2d(pool1, conv2_weights, strides = [1, 1, 1, 1], padding = 'SAME')

        relu2 = tf.nn.relu(tf.nn.bias_add(conv2, conv2_biases))

        pool2 = tf.nn.max_pool(relu2, ksize = [1, 2, 2, 1],
                                strides = [1, 1, 1, 1], padding = 'SAME')

        # Reshape the feature map cuboid into a 2D matrix to feed it to the
        # fully connected layers.
        pool_shape = pool2.get_shape().as_list()
        reshape = tf.reshape(pool2, [pool_shape[0], pool_shape[1] * pool_shape[2] * pool_shape[3]])
        #reshape = tf.Print(reshape, [reshape], "reshape: ", summarize=10)
        # Fully connecte layers
        hidden_layer1 = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases)
        # For training data, add 0.5 dropout. 
        if train:
            hidden_layer1 = tf.nn.dropout(hidden_layer1, 0.5, seed = 0)

        hidden_layer2 = tf.nn. relu(tf.matmul(hidden_layer1, fc2_weights) + fc2_biases)

        if train:
            hidden_layer2 = tf.nn.dropout(hidden_layer2, 0.5, seed = 0)

        return tf.matmul(hidden_layer2, fc3_weights) + fc3_biases

    train_prediction = model_part1(tf_train_x, True)
    
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(train_prediction - tf_train_y), 1))
    # Add L2 regularization to loss
    loss += 1e-7 * (tf.nn.l2_loss(fc1_weights) + tf.nn.l2_loss(fc1_biases) +
            tf.nn.l2_loss(fc2_weights) + tf.nn.l2_loss(fc2_biases) + 
            tf.nn.l2_loss(fc3_weights) + tf.nn.l2_loss(fc3_biases)) 

    # Optimizer
    global_step = tf.Variable(0, name = 'global_step',trainable = False)
    starter_learning_rate = 0.001
    learning_rate = tf.train.exponential_decay(starter_learning_rate,
                                                global_step,
                                                100000,
                                                0.96,
                                                staircase = True)

    optimizer = tf.train.AdamOptimizer(learning_rate, 0.95).minimize(loss, global_step = global_step)

    
    

In [ ]:
num_steps =11

with tf.Session(graph=deep_grap) as sess:
    tf.initialize_all_variables().run()
    print ("TF graph variables initialized ... ")
    
    for step in range(num_steps):
        offset = (step * batch_size) % (y_train.shape[0] - batch_size)

        batch_data = x_train[offset:(offset + batch_size),:]
        batch_labels = y_train[offset:(offset + batch_size)]

        feed_dict = {tf_train_dataset: batch_data,
                     tf_train_labels: batch_labels}

        _,l, pred = sess.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        
        print ("Loss %d: %f" % (step, l))